In [146]:
import pandas as pd
import numpy as np

data = pd.read_csv("新竹_2019.csv", encoding="big5")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6481 entries, 0 to 6480
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   測站                               6481 non-null   object
 1   日期                               6481 non-null   object
 2   測項                               6481 non-null   object
 3   00                               6481 non-null   object
 4   01                               6481 non-null   object
 5   02                               6481 non-null   object
 6   03                               6481 non-null   object
 7   04                               6481 non-null   object
 8   05                               6481 non-null   object
 9   06                               6481 non-null   object
 10  07                               6481 non-null   object
 11  08                               6481 non-null   object
 12  09                               6

In [ ]:
1. 資料前處理

 a. 取出10.11.12月資料

 b. 缺失值以及無效值以前後一小時平均值取代 (如果前一小時仍有空值，再取更前一小時)
  # 表示儀器檢核為無效值，* 表示程式檢核為無效值，x 表示人工檢核為無效值，A 係指因儀器疑似故障警報所產生的無效值，空白 表示缺值。

 c. NR表示無降雨，以0取代

 d. 將資料切割成訓練集(10.11月)以及測試集(12月)

 e. 製作時序資料: 將資料形式轉換為行(row)代表18種屬性，欄(column)代表逐時數據資料

     **hint: 將訓練集每18行合併，轉換成維度為(18,61*24)的DataFrame(每個屬性都有61天*24小時共1464筆資料)

2. 時間序列: 請使用10和11月資料當作訓練集，12月之資料當作測試集，

    將前六小時的汙染物數據做為特徵，未來第一個小時/未來第六個小時的pm2.5數據為預測目標

  a.預測目標

     1. 將未來第一個小時當預測目標

         取6小時為一單位切割，例如第一筆資料為第0~5小時的資料(X[0])，去預測第6小時(未來第一小時)的PM2.5值(Y[0])，下一筆資料為第1~6小時的資料(X[1])去預測第7 小時的PM2.5值(Y[1])  *hint: 切割後X的長度應為1464-6=1458

     2. 將未來第六個小時當預測目標

         取6小時為一單位切割，例如第一筆資料為第0~5小時的資料(X[0])，去預測第11小時(未來第六小時)的PM2.5值(Y[0])，下一筆資料為第1~6小時的資料(X[1])去預測第12 小時的PM2.5值(Y[1])  *hint: 切割後X的長度應為1464-11=1453

 b. X請分別取

     1. 只有PM2.5 (e.g. X[0]會有6個特徵，即第0~5小時的PM2.5數值)

     2. 所有18種屬性 (e.g. X[0]會有18*6個特徵，即第0~5小時的所有18種屬性數值)

 c. 使用兩種模型 Linear Regression 和 Random Forest Regression 建模

 d. 用測試集資料計算MAE (會有8個結果， 2種X資料 * 2種Y資料 * 2種模型)


In [174]:
data.head()

,測站,日期,測項,00,01,02,03,04,05,06,...,14,15,16,17,18,19,20,21,22,23
0,--------------------,--------------------,--------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,...,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------
1,新竹,2019/01/01 00:00:00,AMB_TEMP,17.5,17.4,17.3,17.1,16.8,16.9,16.8,...,20,19.5,18.7,18.1,17.9,17.6,17.6,17.5,17.7,17.5
2,新竹,2019/01/01 00:00:00,CH4,1.79,1.78,1.8,1.8,1.8,1.8,1.8,...,1.81,1.81,1.81,1.8,1.8,1.81,1.81,1.8,1.8,1.79
3,新竹,2019/01/01 00:00:00,CO,0.19,0.21,0.22,0.22,0.21,0.21,0.23,...,0.29,0.3,0.31,0.31,0.31,0.3,0.29,0.27,0.26,0.24
4,新竹,2019/01/01 00:00:00,NMHC,0.04,0.04,0.04,0.04,0.04,0.04,0.04,...,0.06,0.06,0.07,0.07,0.07,0.06,0.05,0.04,0.04,0.04


In [147]:
# data.columns 中有太多空隔，先strip

def clean_col(col):
    col = col.strip()
    return col

new_columns = []
for c in data.columns:
    clean_c = clean_col(c)
    new_columns.append(clean_c)
new_columns
data.columns = new_columns
# Index(['測站', '日期', '測項', '00', '01', '02', '03', '04', '05', '06', '07', '08',
#        '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
#        '21', '22', '23'],
#       dtype='object')

# 對整筆資料集 strip ，清除多餘的空格
for j in range(0,27):
    for i in range(0,len(data[data.columns[j]])):
        data[data.columns[j]][i] =data[data.columns[j]][i].strip()      
        
data = data.replace("NR",0) 
# #NR表示無降雨，以0替代

In [148]:
"""
 a. 取出10.11.12月資料,製成時間序列
 
 10.11月是train_data
 12月是test_data
"""
print("18種*31天=",18*31)
data_dec = data[data["日期"]>= '2019/12/01']
data_dec

data_oct_nov = data[data["日期"]>= '2019/10/01']
data_oct_nov = data_oct_nov[data_oct_nov["日期"] <= '2019/11/31']
print("(31+30)*18=",(31+30)*18)
data_oct_nov

"""
train_data
test_data
轉成list
"""
train18_data = data_oct_nov.drop(['測站', '日期',"測項"], axis=1)
test18_data = data_dec.drop(['測站', '日期',"測項"], axis=1)
test18_data
print(type(test18_data))

18種*31天= 558
(31+30)*18= 1098


,測站,日期,測項,00,01,02,03,04,05,06,...,14,15,16,17,18,19,20,21,22,23
4825,新竹,2019/10/01 00:00:00,AMB_TEMP,24.7,25.1,25.4,25.5,25.3,25.1,24.6,...,32.4,31.9,30.4,29.1,28.7,28.3,28,27.4,27,26.5
4826,新竹,2019/10/01 00:00:00,CH4,1.66,1.66,1.7,1.71,1.72,1.71,1.75,...,1.69,1.72,1.74,1.74,1.78,1.82,1.82,1.83,1.93,1.96
4827,新竹,2019/10/01 00:00:00,CO,0.05,0.13,0.15,0.17,0.16,0.16,0.22,...,0.27,0.32,0.36,0.39,0.49,0.57,0.58,0.6,0.69,0.49
4828,新竹,2019/10/01 00:00:00,NMHC,0,0,0,0.02,0.02,0.01,0.03,...,0.07,0.1,0.08,0.08,0.13,0.21,0.22,0.21,0.22,0.17
4829,新竹,2019/10/01 00:00:00,NO,0,0.3,0.3,0.3,0.3,0.3,1.2,...,1.6,1.1,0.8,0.6,0.6,0.7,0.6,2,5.1,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918,新竹,2019/11/30 00:00:00,THC,1.84,1.82,1.84,1.84,1.91,1.97,1.9,...,1.83,1.94,1.95,1.94,2.04,2.23,2.2,2,2.05,1.94
5919,新竹,2019/11/30 00:00:00,WD_HR,56,65,67,66,95,96,41,...,281,271,257,246,213,165,198,162,167,168
5920,新竹,2019/11/30 00:00:00,WIND_DIREC,53,70,98,20,94,83,39,...,258,273,244,167,178,162,225,170,115,179
5921,新竹,2019/11/30 00:00:00,WIND_SPEED,1.7,1.7,0.8,0.7,0.9,1.3,1.3,...,2.1,1.7,1.3,0.8,0.9,1.2,0.8,0.9,0.9,1


In [150]:
"""
 b. 缺失值以及無效值以前後一小時平均值取代 (如果前一小時仍有空值，再取更前一小時)
  # 表示儀器檢核為無效值，* 表示程式檢核為無效值，x 表示人工檢核為無效值，A 係指因儀器疑似故障警報所產生的無效值，空白 表示缺值。
"""
import re
pattern = r"[*#xA\s]"

# train18_data去無效值&空值
for rows in range(len(train18_data)):
    train_rows = train18_data.iloc[rows]
    for i in range(len(train_rows)):
        j=i-1
        k=i+1
        if re.search(pattern,str(train_rows[i])) or pd.isna(str(train_rows[i])) == True:
            while re.search(pattern,str(train_rows[j])) or pd.isna(str(train_rows[j])) == True:
                j -=1
                while re.search(pattern,str(train_rows[j])) or pd.isna(str(train_rows[j])) == True:
                    j -=1
            while re.search(pattern,str(train_rows[k])) or pd.isna(str(train_rows[k])) == True:
                k +=1
                while re.search(pattern,str(train_rows[k])) or pd.isna(str(train_rows[k])) == True:
                    k +=1
            train18_data.iloc[rows][i]=(float(train_rows[j]) + float(train_rows[k]))/2


# test18_data去無效值&空值
for rows in range(len(test18_data)):
    train_rows = test18_data.iloc[rows]
    for i in range(len(train_rows)):
        j=i-1
        k=i+1
        if re.search(pattern,str(train_rows[i])) or pd.isna(str(train_rows[i])) == True:
            while re.search(pattern,str(train_rows[j])) or pd.isna(str(train_rows[j])) == True:
                j -=1
                while re.search(pattern,str(train_rows[j])) or pd.isna(str(train_rows[j])) == True:
                    j -=1
            while re.search(pattern,str(train_rows[k])) or pd.isna(str(train_rows[k])) == True:
                k +=1
                while re.search(pattern,str(train_rows[k])) or pd.isna(str(train_rows[k])) == True:
                    k +=1
            test18_data.iloc[rows][i]=(float(train_rows[j]) + float(train_rows[k]))/2
    

In [152]:
"""
 e. 製作時序資料: 將資料形式轉換為行(row)代表18種屬性，欄(column)代表逐時數據資料

     **hint: 將訓練集每18行合併，轉換成維度為(18,61*24)的DataFrame(每個屬性都有61天*24小時共1464筆資料)
"""
train18_list=[]
test18_list=[]

#共有18種汙染物
for i in range(18):
    train18_list.append([]) # 創造 18 rows,等等填入18種汙染物
    test18_list.append([])

#製作train18_list
for i in range(0,len(train18_data)):
    for j in range(0,24):
        train18_list[i%18].append(float(train18_data.iloc[i][j]))

#製作test_list
for i in range(0,len(test18_data)):
    for j in range(0,24):
        test18_list[i%18].append(float(test18_data.iloc[i][j]))    
        
print('目前資料len=',len(test18_list[9]))

print(test18_list[9])#pm2.5

目前資料len= 744
[1.86, 1.91, 1.89, 1.87, 1.96, 1.86, 1.86, 1.9, 1.81, 1.76, 1.8, 1.77, 1.73, 1.72, 1.73, 1.74, 1.75, 1.78, 1.77, 1.75, 1.77, 1.81, 1.79, 1.79, 1.78, 1.78, 1.81, 1.81, 1.81, 1.82, 1.83, 1.84, 1.84, 1.83, 1.83, 1.82, 1.82, 1.82, 1.82, 1.82, 1.83, 1.83, 1.81, 1.8, 1.79, 1.79, 1.76, 1.76, 1.77, 1.78, 1.82, 1.83, 1.83, 1.82, 1.83, 1.85, 1.83, 1.81, 1.815, 1.82, 1.81, 1.81, 1.81, 1.81, 1.81, 1.81, 1.82, 1.82, 1.81, 1.81, 1.8, 1.8, 1.8, 1.79, 1.77, 1.77, 1.77, 1.77, 1.76, 1.75, 1.75, 1.75, 1.74, 1.74, 1.73, 1.74, 1.75, 1.76, 1.78, 1.79, 1.8, 1.8, 1.79, 1.79, 1.78, 1.78, 1.79, 1.79, 1.82, 1.81, 1.78, 1.78, 1.78, 1.79, 1.78, 1.78, 1.78, 1.79, 1.78, 1.78, 1.77, 1.77, 1.79, 1.8, 1.8, 1.79, 1.79, 1.8, 1.81, 1.79, 1.78, 1.78, 1.76, 1.76, 1.76, 1.76, 1.75, 1.76, 1.76, 1.75, 1.76, 1.76, 1.78, 1.78, 1.75, 1.77, 1.77, 1.77, 1.76, 1.75, 1.75, 1.75, 1.75, 1.74, 1.74, 1.74, 1.75, 1.76, 1.77, 1.77, 1.78, 1.8, 1.8, 1.79, 1.8, 1.79, 1.78, 1.78, 1.78, 1.78, 1.78, 1.79, 1.78, 1.77, 1.78, 1.77, 1.7

In [156]:
print(test18_list[9]) #pm2.5

[18.0, 18.0, 18.0, 16.0, 18.0, 18.0, 15.0, 26.0, 21.0, 23.0, 19.0, 23.0, 21.0, 17.0, 20.0, 21.0, 17.0, 22.0, 19.0, 19.0, 18.0, 15.0, 15.0, 12.0, 14.0, 11.0, 12.0, 14.0, 13.0, 14.0, 15.0, 11.0, 10.0, 11.0, 12.0, 11.0, 10.0, 10.0, 17.0, 16.0, 27.0, 27.0, 26.0, 25.0, 31.0, 26.0, 20.0, 15.0, 17.0, 17.0, 19.0, 20.0, 18.0, 19.0, 22.0, 16.0, 17.0, 14.0, 16.5, 19.0, 14.0, 11.0, 16.0, 18.0, 15.0, 16.0, 17.0, 15.0, 14.0, 14.0, 17.0, 16.0, 12.0, 13.0, 9.0, 12.0, 12.0, 9.0, 6.0, 7.0, 8.0, 11.0, 9.0, 9.0, 10.0, 10.0, 10.0, 15.0, 17.0, 19.0, 18.0, 20.0, 16.0, 17.0, 13.0, 11.0, 15.0, 14.0, 13.0, 12.0, 13.0, 16.0, 11.0, 11.0, 10.0, 10.0, 10.0, 10.0, 7.0, 8.0, 11.0, 6.0, 7.0, 8.0, 6.0, 5.0, 11.0, 7.0, 7.0, 6.0, 7.0, 7.0, 4.0, 5.0, 7.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 2.0, 4.0, 2.0, 1.0, 5.0, 4.0, 3.0, 5.0, 5.0, 1.0, 1.0, 2.0, 3.0, 1.0, 1.0, 1.0, 4.0, 3.0, 2.0, 3.0, 4.0, 4.0, 5.0, 8.0, 10.0, 12.0, 11.0, 10.0, 11.0, 7.0, 6.0, 8.0, 10.0, 7.0, 8.0, 8.0, 7.0, 8.0, 5.0, 4.0, 7.0, 6.0, 5.0, 8.0, 5.0, 4.0, 

In [ ]:
"""
2. 時間序列: 請使用10和11月資料當作訓練集，12月之資料當作測試集，

    將前六小時的汙染物數據做為特徵，未來第一個小時/未來第六個小時的pm2.5數據為預測目標

  a.預測目標

     1. 將未來第一個小時當預測目標

         取6小時為一單位切割，例如第一筆資料為第0~5小時的資料(X[0])，去預測第6小時(未來第一小時)的PM2.5值(Y[0])，
         下一筆資料為第1~6小時的資料(X[1])去預測第7 小時的PM2.5值(Y[1])  *hint: 切割後X的長度應為1464-6=1458

     2. 將未來第六個小時當預測目標

         取6小時為一單位切割，例如第一筆資料為第0~5小時的資料(X[0])，去預測第11小時(未來第六小時)的PM2.5值(Y[0])，
         下一筆資料為第1~6小時的資料(X[1])去預測第12 小時的PM2.5值(Y[1])  *hint: 切割後X的長度應為1464-11=1453

 b. X請分別取

     1. 只有PM2.5 (e.g. X[0]會有6個特徵，即第0~5小時的PM2.5數值)

     2. 所有18種屬性 (e.g. X[0]會有18*6個特徵，即第0~5小時的所有18種屬性數值)
"""

In [205]:
"""
X取只有PM2.5
未來一、六小時當預測目標的 train,test
"""
pm25_train_X=[]
pm25_train_y_1hr=[]
pm25_train_X_6hr=[]
pm25_train_y_6hr=[]
for i in range(len(train18_list[9])-6):
    pm25_train_X.append(train18_list[9][i:i+5])
    pm25_train_y_1hr.append(train18_list[9][i+5])
for i in range(len(train18_list[9])-10):
    pm25_train_X_6hr.append(train18_list[9][i:i+5])  
    pm25_train_y_6hr.append(train18_list[9][i+10])

pm25_test_X=[]
pm25_test_y_1hr=[]
pm25_test_X_6hr=[]
pm25_test_y_6hr=[]
for i in range(len(test18_list[9])-6):
    pm25_test_X.append(test18_list[9][i:i+5])
    pm25_test_y_1hr.append(test18_list[9][i+5])
for i in range(len(test18_list[9])-10):
    pm25_test_X_6hr.append(test18_list[9][i:i+5])  
    pm25_test_y_6hr.append(test18_list[9][i+10])
    
pm25_train_X=np.array(pm25_train_X)
pm25_train_y_1hr=np.array(pm25_train_y_1hr)
pm25_train_X_6hr=np.array(pm25_train_X_6hr)
pm25_train_y_6hr=np.array(pm25_train_y_6hr)
pm25_test_X=np.array(pm25_test_X)
pm25_test_y_1hr=np.array(pm25_test_y_1hr)
pm25_test_X_6hr=np.array(pm25_test_X_6hr)
pm25_test_y_6hr=np.array(pm25_test_y_6hr)
    

In [199]:
print(len(pm25_train_X))
print(len(pm25_train_y_1hr))
print(len(pm25_train_X_6hr))
print(len(pm25_train_y_6hr))

1458
1458
1454
1454


In [222]:
"""
  c. 使用 Linear Regression 建模
  X取只有PM2.5
"""
# LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
# 詳讀 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression
from sklearn.linear_model import LinearRegression

#用0~5小時pm2.5，預測未來一小時pm2.5
regr = LinearRegression() 
regr.fit(pm25_train_X, pm25_train_y_1hr) 
predict_pm25_y1 = regr.predict(pm25_test_X)

MAE_LR = abs(predict_pm25_y1.sum() - pm25_test_y_1hr.sum())
print("用0~5小時pm2.5，Linear Regression預測未來一小時pm2.5算出來的MAE是{}".format(MAE_LR))

#用0~5小時pm2.5，預測未來第六小時pm2.5
regr6 = LinearRegression() 
regr6.fit(pm25_train_X_6hr, pm25_train_y_6hr) 
predict_pm25_y6 = regr6.predict(pm25_test_X_6hr)

MAE_LR_6 = abs(np.array(predict_pm25_y6).sum() - np.array(pm25_test_y_6hr).sum())
print("用0~5小時pm2.5，Linear Regression預測未來第六小時pm2.5算出來的MAE是{}".format(MAE_LR_6))

用0~5小時pm2.5，Linear Regression預測未來一小時pm2.5算出來的MAE是169.636213351896
用0~5小時pm2.5，Linear Regression預測未來第六小時pm2.5算出來的MAE是742.2070926718152


In [224]:
"""
  c. 使用 Random Forest Regression 建模
  X取只有PM2.5
"""
#用0~5小時pm2.5，預測未來一小時pm2.5
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=2, random_state=0)
clf.fit(pm25_train_X, pm25_train_y_1hr)       
y_pred_rf_y1 = clf.predict(pm25_test_X)

MAE_RFR = abs(y_pred_rf_y1.sum() - pm25_test_y_1hr.sum())
print("用0~5小時pm2.5，Random Forest Regression預測未來一小時pm2.5算出來的MAE是{}".format(MAE_RFR))

#用0~5小時pm2.5，預測未來第六小時pm2.5
clf6 = RandomForestRegressor(max_depth=2, random_state=0)
clf6.fit(pm25_train_X_6hr, pm25_train_y_6hr)       
y_pred_rf_y6 = clf6.predict(pm25_test_X_6hr)

MAE_RFR_6 = abs(np.array(predict_pm25_y6).sum() - np.array(pm25_test_y_1hr).sum())
print("用0~5小時pm2.5，Random Forest Regression預測未來第六小時pm2.5算出來的MAE是{}".format(MAE_RFR_6))

用0~5小時pm2.5，Random Forest Regression預測未來一小時pm2.5算出來的MAE是498.49887401265914
用0~5小時pm2.5，Random Forest Regression預測未來第六小時pm2.5算出來的MAE是650.2070926718152


In [202]:
"""
X取所有18種屬性
未來一、六小時當預測目標的 train,test

"""
pollut18_train_X=[]
pollut18_train_y_1hr=[]
pollut18_train_X_6hr=[]
pollut18_train_y_6hr=[]
for i in range(len(train18_list[9])-6):
    pollut18_train_X.append(train18_list[i*(18):(i+1)*(18)][i:i+5])
    pollut18_train_y_1hr.append(train18_list[9][i+5])
for i in range(len(train18_list[9])-10):
    pollut18_train_X_6hr.append(train18_list[i*(18):(i+1)*(18)][i:i+5])
    pollut18_train_y_6hr.append(train18_list[9][i+10])

pollut18_test_X=[]
pollut18_test_y_1hr=[]
pollut18_test_X_6hr=[]
pollut18_test_y_6hr=[]
for i in range(len(test18_list[9])-6):
    pollut18_test_X.append(test18_list[i*(18):(i+1)*(18)][i:i+5])
    pollut18_test_y_1hr.append(test18_list[9][i+5])
for i in range(len(test18_list[9])-10):
    pollut18_test_X_6hr.append(test18_list[i*(18):(i+1)*(18)][i:i+5])
    pollut18_test_y_6hr.append(test18_list[9][i+10])
        
pollut18_train_X=np.array(pollut18_train_X)
pollut18_train_y_1hr=np.array(pollut18_train_y_1hr)
pollut18_train_X_6hr=np.array(pollut18_train_X_6hr)
pollut18_train_y_6hr=np.array(pollut18_train_y_6hr)
pollut18_test_X=np.array(pollut18_test_X)
pollut18_test_y_1hr=np.array(pollut18_test_y_1hr)
pollut18_test_X_6hr=np.array(pollut18_test_X_6hr)
pollut18_test_y_6hr=np.array(pollut18_test_y_6hr)

In [ ]:
from sklearn.linear_model import Ridge,RidgeCV
#使用广义交叉验证 拟合得到最优alpha参数 = 898.21
regs = RidgeCV(np.linspace(1,1000))
regs.fit(pollut18_train_X, pollut18_train_y_1hr)

alpha = regs.alpha_

In [216]:
"""
  c. 使用 Linear Regression建模
  X取18種汙染物
  
  X取18種汙染物的系列，
  一直出現error "setting an array element with a sequence."
  不知道怎麼更正 啊啊啊啊啊
"""
# LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
# 詳讀 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

from sklearn.linear_model import Ridge


#用0~5小時pm2.5，預測未來一小時pm2.5
reg = Ridge(alpha=898.21)
reg.fit(pollut18_train_X, pollut18_train_y_1hr) 
predict_18_y1 = reg.predict(pollut18_test_X_1hr)

#絕對值函數abs()
MAE_LR = abs(predict_18_y1.sum() - pollut18_test_y_1hr.sum())
print("用0~5小時pm2.5，預測未來一小時pm2.5算出來的MAE是{}".format(MAE_LR))

ValueError: setting an array element with a sequence.

In [213]:
"""
  c. 使用 Random Forest Regression 建模
  X取18種汙染物
"""
#用0~5小時pm2.5，預測未來一小時pm2.5
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=2, random_state=0)
clf.fit(pollut18_train_X, pollut18_train_y_1hr)       
pred18_rf_y1 = clf.predict(pollut18_test_X_1hr)

MAE_18RFR = abs(pred18_rf_y1.sum() - pollut18_test_y_1hr.sum())
print("用0~5小時pm2.5，預測未來一小時pm2.5算出來的MAE是{}".format(MAE_18RFR))

#用0~5小時pm2.5，預測未來第六小時pm2.5
clf6 = RandomForestRegressor(max_depth=2, random_state=0)
clf6.fit(pollut18_train_X_6hr, pollut18_train_y_6hr)       
pred_18rf_y6 = clf6.predict(pollut18_test_X_6hr)

MAE_18RFR_6 = abs(pred_18rf_y6.sum() - pollut18_train_y_6hr.sum())
print("用0~5小時pm2.5，預測未來第六小時pm2.5算出來的MAE是{}".format(MAE_18RFR_6))

ValueError: setting an array element with a sequence.